## Binomial Option Pricing

Implementing a Binomial Pricing Models in few ways.

In [3]:
#Imports
import numpy as np


## Binomial Tree Representation

The Stock tree can be represented using nodes (i, j) and initial stock price $S_{i,j} = S_{0}u^jd^{i-j}$



$C_{i,j}$ represents a final payoff function that we can define. $C_{N,j}$ represents the contract price at each node (i, j). 
Here we are pricing a European Call 
$C_{N, j} = max(S_{N, j} - K,0)$

In [22]:
# Parameters
S0 = 100 # Initial Stock Price
K = 100 # Strike Price
T = 1 # Time to Maturity
r = 0.05 # Annual Risk-free rate
N = 3 # Number of time steps
u = 1.1 # Up factor in binomial model
d = 1/u # ensure binomial tree recombines
option_type = 'C' #C or P for call or put

# Slow implementation of Binomial Tree

In [23]:
def slow_binomial_tree(S0, K, T, r, N, u, d):
    #precompute constants
    dt = T/N #This is the time step
    q = (np.exp(r*dt)-d) / (u-d) #Risk-neutral probability of up movement
    discount = np.exp(-r*dt) #Discount factor

    #Initialize asset prices at maturity at t = N

    S = np.zeros(N+1)
    S[0] = S0 * d**N #Price at the bottom of the binomial tree at the end
    for j in range(1, N+1):
        S[j] = S[j-1] * u/d 
        #Dividing by d allows us to go back 1 step
        #Multiplying by u allows us to go up 1 step

    #Initialize option payoffs at maturity
    C = np.zeros(N+1)
    for j in range(0, N+1):
        C[j] = max(S[j]-K, 0)
    
    # Step back through the tree

    for i in np.arange(N, 0, -1):
        for j in range(0, i):
            C[j] = discount * (q * C[j+1] + (1-q) *C[j])
            #This is the binomial tree formula
    
    return C[0]


In [24]:
slow_binomial_tree(S0, K, T, r, N, u, d)

9.614332823683377

# Fast Implementation of Binomial Tree

In [25]:
def fast_binomial_tree(S0, K, T, r, N, u, d):
    #precompute constants
    dt = T/N #This is the time step
    q = (np.exp(r*dt)-d) / (u-d) #Risk-neutral probability of up movement
    discount = np.exp(-r*dt) #Discount factor

    #Initialize asset prices at maturity at t = N

    C = S0 * d**(np.arange(N, -1, -1)) * u**(np.arange(N+1))

    #Initialize option payoffs at maturity at t = N

    C = np.maximum(C - K, 0)
    
    # Step back through the tree



    for i in np.arange(N, 0, -1):
        C = discount * (q * C[1:i+1] + (1-q) *C[0:i])
            #This is the binomial tree formula
    
    return C[0]
fast_binomial_tree(S0, K, T, r, N, u, d)

9.614332823683386

Here let's price an American Option

# American Option Characteristics
For an American Put Option
if $T = t_N$ then at the terminal nodes, $C^{j}_N = (K-S^{j}_N)^{+}$

for all other parts of the tree at nodes $(i,j)$

- Max of exercise value or continuation/hold value

- $C^{j}_i = \max \Big((K-S^{j}_i)^{+}, \exp^{-r\Delta t} \big\{ q^{j}_i C^{j+1}_{i+1} + (1 - q^{j}_i)C^{j-1}_{i-1}\big\}\Big)$

In [33]:
# Initialize Parameters
S0 = 100 # Initial Stock Price
K = 100 # Strike Price
T = 1 # Time to Maturity
r = 0.06 # Annual Risk-free rate
N = 3 # Number of time steps
u = 1.1 # Up factor in binomial model
d = 1/u # ensure binomial tree recombines
option_type = 'P' #C or P for call or put

In [34]:
def slow_american_tree(S0, K, T, r, N, u, d, option_type):
    #precompute constants
    dt = T/N #This is the time step
    q = (np.exp(r*dt)-d) / (u-d) #Risk-neutral probability of up movement
    discount = np.exp(-r*dt) #Discount factor

    #Initialize asset prices at maturity at t = N

    S = np.zeros(N+1)
    for j in range(0, N+1):
        S[j] = S0 * u**j * d**(N-j)
    
    #Initialize option payoffs at maturity

    C = np.zeros(N+1)
    for j in range(0, N+1):
        if option_type == 'C':
            C[j] = max(S[j]-K, 0)
        else:
            C[j] = max(K-S[j], 0)

    # Step back through the tree
    for i in np.arange(N-1, -1, -1):
        for j in range(0, i+1):
            S = S0 * u**j * d**(i-j)
            C[j] = discount * (q * C[j+1] + (1-q) *C[j])
            if option_type == 'C':
                C[j] = max(C[j], S-K)
            else:
                C[j] = max(C[j], K-S)
        
    return C[0]

slow_american_tree(S0, K, T, r, N, u, d, option_type)

4.654588754602527

In [36]:
def fast_american_tree(S0, K, T, r, N, u, d, option_type):
    #precompute constants
    dt = T/N #This is the time step
    q = (np.exp(r*dt)-d) / (u-d) #Risk-neutral probability of up movement
    discount = np.exp(-r*dt) #Discount factor

    #Initialize asset prices at maturity at t = N

    S = S0 * d**(np.arange(N, -1, -1)) * u**(np.arange(N+1))
    
    #Initialize option payoffs at maturity


    if option_type == 'C':
        C = np.maximum(S-K, 0)
    else:
        C = np.maximum(K-S, 0)

    # Step back through the tree
    for i in np.arange(N-1, -1, -1):
        S = S0 * d**(np.arange(i, -1, -1)) * u**(np.arange(i+1))
        C[:i+1] = discount * (q * C[1:i+2] + (1-q) *C[0:i+1])
        C = C[:-1]
        if option_type == 'C':
            C = np.maximum(C, S-K)
        else:
            C = np.maximum(C, K-S)
    
    return C[0]
fast_american_tree(S0, K, T, r, N, u, d, option_type)

4.654588754602527

## Black Scholes Option Pricing

In [37]:
import numpy as np
from scipy.stats import norm

In [38]:
#Define Variables
r = 0.05 # Risk-free rate
S0 = 100 # Initial Stock Price
K = 100 # Strike Price
T = 240/365 # Time to Maturity\
sigma = 0.2 # Volatility

In [39]:
def blackScholes(r, S, K, T, sigma, option_type):
    #Calculate Black Scholes price for call or put option

    d1 = (np.log(S/K) + (r + (sigma**2)/2) * T) / (sigma * np.sqrt(T)) 
    d2 = d1 - sigma*np.sqrt(T)

    if option_type == 'C':
        price = S * norm.cdf(d1, 0, 1) - K * np.exp(-r * T) * norm.cdf(d2, 0, 1)
    elif option_type == 'P':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

blackScholes(r, S0, K, T, sigma, 'C')

8.105343276254992

# Monte Carlo Simulation